In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import explained_variance_score,mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.experimental import Adamax, Adadelta, Adagrad, Ftrl, Nadam, RMSprop, SGD
from keras.models import load_model
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/weather_2017+2018_hourly.csv")
data_as_np = dataset.GHI.values

In [ ]:
def series_to_supervised(data_np,n_lags,n_outs):
    samples = len(data_np)-(n_lags+n_outs-1)
    Inputs = []
    Targets = np.empty((samples,n_outs))
    for i in range(samples):
        step = i+n_lags
        sample = data_np[i:step]
        Inputs.append(sample)
        label = data_np[step:(step+n_outs)]
        j = 0
        for a in label:
            Targets[i,j] = a
            j = j+1
    return np.array(Inputs),Targets

In [ ]:
def plot_model_history(model_summary):
    plt.plot(model_summary.history['loss'])
    plt.plot(model_summary.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'],loc='upper right')
    plt.show()

In [ ]:
data_np = data_as_np.reshape(-1,1)
def prepare_model(n_lags):
  n_out = 24

  # Prepare data
  Inputs,Targets = series_to_supervised(data_np,n_lags,n_out)

  Inputs_model, Targets_model = Inputs[:-10], Targets[:-10]
  Inputs_plot, Targets_plot = Inputs[-10:], Targets[-10:]

  no_samples = Targets_model.shape[0]
  split = int(0.15 * no_samples)

  # Splitting data into train and test data set
  Inputs_model = Inputs_model.reshape((no_samples,n_lags,1))

  train_x,train_y = Inputs_model[:-split],Targets_model[:-split]
  test_x,test_y = Inputs_model[-split:],Targets_model[-split:]
  return train_x,train_y,test_x,test_y

In [ ]:
def rmse_error(pred,y):
    return np.sqrt(mean_squared_error(pred,y))

In [ ]:
def create_model(input_shape,n_out,train_x,train_y,test_x,test_y):
    # define model
    inputs = Input(shape = input_shape)
    # input layer...
    lstm1 = LSTM(64, activation = 'relu')(inputs)
    #dr1 = Dropout(0.1)(lstm1)
    #lstm2 = LSTM(64, activation = 'relu')(lstm1)
    #dr2 = Dropout(0.1)(lstm1)(lstm2)
    #output layer...
    output = Dense(n_out,activation = 'relu')(lstm1)
    model = Model(inputs = inputs, outputs = output)

    #cp1 = ModelCheckpoint('StackLSTM/', save_best_only=True)
    model.compile(loss='mean_squared_error', optimizer = Adam(0.0001), metrics=[RootMeanSquaredError()])
    model_summary = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=70, batch_size=64) #callbacks=[cp1]
    plot_model_history(model_summary)

    #model = load_model('StackLSTM/')

    return model

In [ ]:
train_rmse_error, test_rmse_error, test_mae = 0,0,0
data_np = data_as_np.reshape(-1,1)
n_lags = 60
n_out = 24
# Prepare data
Inputs,Targets = series_to_supervised(data_np,n_lags,n_out)

Inputs_model, Targets_model = Inputs[:-10], Targets[:-10]
Inputs_plot, Targets_plot = Inputs[-10:], Targets[-10:]

no_samples = Targets_model.shape[0]
split = int(0.3 * no_samples)

input_shape = (n_lags,1)

train_x,train_y = Inputs_model[:-split],Targets_model[:-split]
test_x,test_y = Inputs_model[-split:],Targets_model[-split:]
for _ in range(4):
  model = create_model(input_shape,n_out,train_x,train_y,test_x,test_y)

  train_pred,test_pred = model.predict(train_x), model.predict(test_x)

  train_rmse_error = rmse_error(train_pred.flatten(),train_y.flatten())
  test_rmse_error = rmse_error(test_pred.flatten(),test_y.flatten())
  test_mae = mean_absolute_error(test_pred.flatten(),test_y.flatten())
  with open("/content/drive/MyDrive/LSTMerror1.txt","a") as f:
    f.write(f"{train_rmse_error},{test_rmse_error},{test_mae}\n")